In [1]:
# Dependencias.
import scipy.io
from preprocessing_functions import read_rhd
%matplotlib widget
import h5py
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import probeinterface as pi
import spikeinterface as si
import spikeinterface.widgets as sw
import spikeinterface.sorters as ss
import spikeinterface.exporters as exp
import seaborn as sns # <- Para graficos estadisticos
from preprocessing_functions import read_rhd, get_recording, check_concatenation, process_artifacts, espigas, sorting_analyzer, create_folders

from pathlib import Path
from spikeinterface.extractors import read_intan
import spikeinterface.preprocessing as prep
from probeinterface import Probe, ProbeGroup

# global kwargs for parallel computing
job_kwargs = dict(
    n_jobs=-1,
    chunk_duration='1s',
    progress_bar=True,
)

si.set_global_job_kwargs(**job_kwargs)

17:17:45 [I] klustakwik KlustaKwik2 version 0.2.6


## Lectura y Preprocesado.
el procesado de datos se realiza de la siguiente forma:  
```mermaid
flowchart TD
    A["Datos crudos (Raw)"] --> B["Filtro pasa banda (500 - 9000Hz)"]
    B --> C["Incorporacion de electrodos y definición de grupos"]
    C --> D["Definición de canales malos"]
    D --> E["Remoción de artefactos"]
    E --> F["`Guardado en formato binario (carpeta **preprocess**)`"]
    
```
Algunas opciones comunes de group_mode en SpikeInterface incluyen:

by_probe: Agrupa los canales o unidades según el probe (sonda) al que pertenecen. Esto es útil si se están utilizando varias sondas en el experimento.

by_shank: Agrupa los canales según el shank (tallo) dentro de un probe. Esto se usa cuando un probe tiene múltiples shanks.

by_electrode_group: Agrupa los canales por el grupo de electrodos. Esto permite analizar o procesar datos por grupos predefinidos de electrodos.

by_channel: Trata cada canal por separado, sin agruparlos. Es útil cuando se quiere analizar cada canal de manera independiente.

by_unit: Agrupa los datos según unidades individuales de spikes. Esto es útil cuando se quiere analizar características a nivel de unidades de spikes individuales en lugar de a nivel de canales.

all: Agrupa todos los canales o unidades en un solo grupo, tratándolos como un conjunto único. Esto puede ser útil para obtener un análisis global del conjunto de datos.

### Defición de parametros.
:penci2: para el ejemplo, se almacenan todos los pasos intermedios  :bowtie:

In [14]:
# Nombre base de experimento, se crearán carpetas y archivos con este nombre.
base_name = 'PF11_day02'
artifact_maze = "selected_"
artifact_sleep = 'selected_'
# Archivo de configuración del probegroup, existente en la carpeta "probes"
probegroup_file = 'probes/anillo_probe.json'

# Configuración de carpetas de procesamiento
preprocess_folder = Path('preprocess/',base_name)

# Archivos de Excel para la información de registros
# como inicio se recomienda solo poner un dia en el excel de información de archivos, este es el punto de inicio para hacer una maquina de salchichas para todos los días del animal
folder_maze = Path(r'D:\ephys\PF11\Maze\Day02')
folder_sleep = Path(r'D:\ephys\PF11\Sleep\Day02')

In [15]:
# Definir carpetas de sorting.
# Single sorting
sorter_folder, analyzer_folder, phy_output_folder = create_folders(base_name)
# Group sorting
group_sorter_folder, group_analyzer_folder, group_phy_output_folder = create_folders(base_name, group=True)

# Procesar archivos
record_maze = get_recording(folder_maze, artifact_maze, probegroup_file)
record_sleep = get_recording(folder_sleep, artifact_sleep, probegroup_file)
recording = check_concatenation(record_maze, record_sleep)

Archivo de probegroup utilizado: probes/anillo_probe.json
Procesado 1 archivo .rhd: PF11_Maze_Day02_240508_125024.rhd
Grabación leída con 39817344 muestras y duración total de 1990.8672 segundos. Frecuencia de muestreo calculada: 20000 Hz.
Ruta del archivo LAN: D:\ephys\PF11\Maze\Day02\LAN_PF11_Maze_Day02_240508_125024_500.mat
Dataset encontrado: selected_CA1v_CA1d_RSC_PFC_1_8_10_13
Filtro pasa banda aplicado: 500-9000 Hz.
Probegroup configurado para la grabación.
Artefactos removidos. Triggers utilizados: [     360     2360     4360 ... 39660120 39662120 39664120]
Procesamiento de archivo completado.
Archivo de probegroup utilizado: probes/anillo_probe.json
Procesado 1 archivo .rhd: PF11_Sleep_Day02_240508_134502.rhd
Grabación leída con 86614784 muestras y duración total de 4330.7392 segundos. Frecuencia de muestreo calculada: 20000 Hz.
Ruta del archivo LAN: D:\ephys\PF11\Sleep\Day02\LAN_PF11_Sleep_Day02_240508_134502_500.mat
Dataset encontrado: selected_CA1v_CA1d_RSC_PFC_1_8_10_13
Fi

### Inspeccion de los resultados.
Revise:
Se haya realizado el procedimiento de concatenado.
Se removieron los artefactos.
Nombres de los canales.

importante: Con el fin de mejorar la visualización, los datos no estan escalados. Para presentarlos escalados (uV) defina return_scaled=True

In [16]:
# plot and check spikes
mode = "line"
w = sw.plot_traces(recording=recording,  
                   mode=mode, time_range=(0, 105), color_groups=True, return_scaled=False,
                  show_channel_ids=True, order_channel_by_depth=False, backend="ephyviewer")

recording

c:\ProgramData\anaconda3\envs\sorter\Lib\site-packages\ephyviewer\traceviewer.py:206: RuntimeWarning: divide by zero encountered in divide
  gains[self.visible_channels] = np.ones(nb_visible, dtype=float) / self.signals_mad[self.visible_channels] * self.viewer.params['auto_scale_factor']
c:\ProgramData\anaconda3\envs\sorter\Lib\site-packages\ephyviewer\traceviewer.py:207: RuntimeWarning: invalid value encountered in multiply
  offsets[self.visible_channels] = np.arange(nb_visible)[::-1] - self.signals_med[self.visible_channels]*gains[self.visible_channels]
c:\ProgramData\anaconda3\envs\sorter\Lib\site-packages\ephyviewer\traceviewer.py:140: RuntimeWarning: invalid value encountered in multiply
  return (self.raw_offsets * self.gains) + self.offsets
c:\ProgramData\anaconda3\envs\sorter\Lib\site-packages\ephyviewer\traceviewer.py:605: RuntimeWarning: invalid value encountered in scalar multiply
  self.channel_labels[c].setPos(t_start, offsets[c] + signals_med[c]*gains[c])
c:\ProgramData\

ConcatenateSegmentRecording: 16 channels - 20.0kHz - 1 segments - 126,432,128 samples 
                             6,321.61s (1.76 hours) - int16 dtype - 3.77 GiB

### Eliminacion de canales.
indique los canales a eliminar, para ello utilice el nombre ("Channel ID") que se le da en Intan. Observe el ejemplo.
Confirme que se haya realizado la eliminación ejecutando la celda anterior.

In [17]:
bad_channels =["A-019", "A-023"]
recording=recording.remove_channels(bad_channels)

### Guardar archivos preprocesados
Guardar archivos preprocesados como binarios.

#### Guardar el registro completo

In [ ]:
recording.save(folder=preprocess_folder, overwrite=True, **job_kwargs)
    # rec_artifacts.save(folder=preprocess_artifacts, **job_kwargs) # ejercicio para guardar registro sin la eliminaciòn de artefactos.

#### Guardar un segmento del registro

In [ ]:
sliced_rec=recording.time_slice(start_time=0, end_time=2000)
sliced_rec.save(format='binary', folder=Path('preprocess/slic3d'), overwrite=True, **job_kwargs)

#### Leer el registro guardado

In [ ]:
recording = si.load_extractor(preprocess_folder)

## Ejecutar un sorter
ejecutar Kilosort

### revisar parametros de entrada (configuracion)
parametros especificos para kilosort (pasables por kwargs):  
https://kilosort.readthedocs.io/en/latest/parameters.html  
https://github.com/MouseLand/Kilosort/blob/main/kilosort/parameters.py

In [18]:
# limpiar la memoria de torch antes de procesar el sorter.
import torch
torch.cuda.empty_cache()

ss.get_default_sorter_params('kilosort4')

{'batch_size': 60000,
 'nblocks': 1,
 'Th_universal': 9,
 'Th_learned': 8,
 'do_CAR': True,
 'invert_sign': False,
 'nt': 61,
 'shift': None,
 'scale': None,
 'artifact_threshold': None,
 'nskip': 25,
 'whitening_range': 32,
 'highpass_cutoff': 300,
 'binning_depth': 5,
 'sig_interp': 20,
 'drift_smoothing': [0.5, 0.5, 0.5],
 'nt0min': None,
 'dmin': None,
 'dminx': 32,
 'min_template_size': 10,
 'template_sizes': 5,
 'nearest_chans': 10,
 'nearest_templates': 100,
 'max_channel_distance': None,
 'templates_from_data': True,
 'n_templates': 6,
 'n_pcs': 6,
 'Th_single_ch': 6,
 'acg_threshold': 0.2,
 'ccg_threshold': 0.25,
 'cluster_downsampling': 20,
 'cluster_pcs': 64,
 'x_centers': None,
 'duplicate_spike_ms': 0.25,
 'scaleproc': None,
 'save_preprocessed_copy': False,
 'torch_device': 'auto',
 'bad_channels': None,
 'clear_cache': False,
 'save_extra_vars': False,
 'do_correction': True,
 'keep_good_only': False,
 'skip_kilosort_preprocessing': False,
 'use_binary_file': None,
 'del

## Kilosort 4

In [20]:
params_kilosort4 = {## MAIN_PARAMETERS 
                    'batch_size': 60000,
                    'nblocks': 0,
                    'Th_universal': 7,
                    'Th_learned': 6,
                    ## Preprocessing
                    #'artifact_threshold': 1000,
                    ## SPIKE DETECTION
                    'min_template_size': 10,
                    'template_sizes':5,
                    'nearest_chans': 4,
                    'nearest_templates': 15,
                    'max_channel_distance': 60,
                    'templates_from_data': True,   
                    'n_pcs':10,
                    'Th_single_ch': 4,
                    ## Clustering
                        #'acg_threshold':0.15,
                        #'cluster_downsampling':10,
                    ## extras
                        #'binning_depth':4,
                        #'drift_smoothing':[0.3, 0.3, 0.3],
                    'skip_kilosort_preprocessing': False,} # se crea un diccionario donde se pueden pasar las variables modificadas al sorter.

### Sorting en bulto

In [21]:
sorter = ss.run_sorter(
                sorter_name='kilosort4',
                recording = recording,
                verbose=True,
                folder = sorter_folder,
                remove_existing_folder=True,  ## CUIDADO, SOBREESCRIBE LOS DATOS EN CASO DE HABER UNA CARPETA, PARA DESHABILITAR PONER =FALSE
                **params_kilosort4)

100%|██████████| 2/2 [01:05<00:00, 32.95s/it]


kilosort4 run time 418.96s


In [11]:
num_clusters, total_spikes= espigas(sorter)
print(f"Número total de clusters: {num_clusters}")
print(f"Número total de espigas: {total_spikes}")

Número total de clusters: 31
Número total de espigas: 550603


In [12]:
analyzer=sorting_analyzer(sorter, recording, output_folder=analyzer_folder)

estimate_sparsity:   0%|          | 0/5656 [00:00<?, ?it/s]

compute_waveforms:   0%|          | 0/5656 [00:00<?, ?it/s]

Fitting PCA:   0%|          | 0/31 [00:00<?, ?it/s]

Projecting waveforms:   0%|          | 0/31 [00:00<?, ?it/s]

Compute : spike_amplitudes:   0%|          | 0/5656 [00:00<?, ?it/s]

In [13]:
exp.export_to_phy(sorting_analyzer=analyzer, 
                  remove_if_exists=True, 
                  copy_binary=True, 
                  output_folder=phy_output_folder)

write_binary_recording:   0%|          | 0/5656 [00:00<?, ?it/s]

extract PCs:   0%|          | 0/5656 [00:00<?, ?it/s]

Run:
phy template-gui  C:\Users\Labcn\sorter\SorterPipeline\output\AN\kilosort\phy_PF11_day01\params.py


## revision datos ariel

In [ ]:
import spikeinterface.extractors as ex
import spikeinterface.extractors as se

# Definir la configuración del archivo
file_path = Path(r'F:\AL\11-12-24\phy_TM37_d1_test\recording.dat')  # Ruta al archivo .dat
num_channels = 61  # Número de canales en el archivo
sampling_frequency = 30000  # Frecuencia de muestreo en Hz
dtype = "int16"  # Tipo de datos, por ejemplo: "int16", "float32", etc.
# channel_ids = list(range(num_channels))  # IDs de los canales (opcional)

# Crear el extractor
recording = se.BinaryRecordingExtractor(
    file_paths=file_path,
    sampling_frequency=sampling_frequency,
    num_channels=num_channels,
    dtype=dtype,
    # channel_ids=channel_ids
)

probegroup_file = 'probes/TM24_probe.json'
probegroup = pi.read_probeinterface(probegroup_file)
recording = recording.set_probegroup(probegroup, group_mode='by_probe')

# Imprimir detalles de la grabación
print(f"Número de canales: {recording.get_num_channels()}")
print(f"Frecuencia de muestreo: {recording.get_sampling_frequency()} Hz")
print(f"Duración: {recording.get_num_samples() / recording.get_sampling_frequency()} segundos")

#phy reader
phy_output_folder = Path(r'F:\AL\11-12-24\sorter_TM37_d1_test\sorter_output')
readed_sorter = ex.read_phy(phy_output_folder, exclude_cluster_groups=None)


In [ ]:
analyzer=sorting_analyzer(readed_sorter, recording, output_folder="")

In [ ]:
available_extension_names = analyzer.get_loaded_extension_names()
print(available_extension_names)

### Sorting por grupo : Automatic splitting

In [ ]:
group_sorter = ss.run_sorter_by_property(
                sorter_name='kilosort4',
                recording = recording,
                grouping_property='group',
                verbose=True,
                folder = group_sorter_folder,
                remove_existing_folder=True,  ## CUIDADO, SOBREESCRIBE LOS DATOS EN CASO DE HABER UNA CARPETA, PARA DESHABILITAR PONER =FALSE
                **params_kilosort4)

num_clusters, total_spikes= espigas(group_sorter)
print(f"Número total de clusters: {num_clusters}")
print(f"Número total de espigas: {total_spikes}")

group_analyzer=sorting_analyzer(group_sorter, recording, output_folder=group_analyzer_folder)

exp.export_to_phy(sorting_analyzer=group_analyzer, 
                  remove_if_exists=True, 
                  copy_binary=True, 
                  output_folder=group_phy_output_folder) 

### Sorting por grupo : Manual splitting (en desarrollo)

El enfoque implementado tiene como objetivo mejorar el rendimiento del spike sorting al segmentar el proceso por grupos de canales o regiones de la grabación. Al realizar el spike sorting por partes, se reduce significativamente la demanda de memoria y el uso de GPU en cada ejecución. Esto es especialmente útil cuando se trabaja con grabaciones de gran tamaño o cuando la capacidad de la GPU es limitada.

Dividiendo el registro en grupos, el sorter puede operar en fragmentos más pequeños, lo que permite manejar mejor los recursos del sistema y evitar problemas como errores de "out of memory" (falta de memoria). Después de realizar el spike sorting por grupos, los resultados se combinan, permitiendo un análisis y exportación global sin sacrificar la eficiencia durante el proceso de clasificación inicial.

Este enfoque balancea el uso de recursos, optimizando el uso de la GPU sin comprometer la calidad del análisis final.

In [ ]:
# Dividir la grabación por grupos
split_recording = recording.split_by("group")

# Diccionario para almacenar los resultados de sorting por grupo
sortings = {}

# Ejecutar el sorter en cada grupo
for group, sub_recording in split_recording.items():
    sorting = run_sorter(
        sorter_name='kilosort4',
        recording=sub_recording,  # Usar la subgrabación del grupo
        output_folder=f"fKS4_group{group}"
    )
    sortings[group] = sorting  # Almacenar los resultados del sorter para este grupo

# Combinar los resultados de sorting de todos los grupos
combined_sorting = si.concatenate_sortings(*sortings.values())

# Información de clusters y espigas para cada grupo
num_clusters, total_spikes = espigas(combined_sorting)
print(f"  Número total de clusters: {num_clusters}")
print(f"  Número total de espigas: {total_spikes}")

# Realizar análisis global con el sorting combinado
combined_analyzer = sorting_analyzer(combined_sorting, recording, output_folder='combined_analyzer_folder')

# Exportar los resultados globales a Phy
exp.export_to_phy(sorting=combined_analyzer,  # Sorting combinado
                  recording=recording,        # Grabación completa
                  remove_if_exists=True,
                  copy_binary=True,
                  output_folder='combined_phy_output_folder')


## inspeccion en grafico

### leer un analisis

In [ ]:

folder = 'output/AN/kilosort/analyzer_Rev9'
analyzer = si.load_sorting_analyzer(folder)

In [ ]:
exp.export_to_phy(sorting_analyzer=analyzer, 
                  remove_if_exists=True, 
                  copy_binary=True, 
                  output_folder=Path('output/AN/kilosort/phy_rev9v2'))

In [ ]:
import spikeinterface.widgets as sw

sw.plot_spikes_on_traces(sorting_analyzer= analyzer, 
                         segment_index=None, 
                         channel_ids=None, 
                         unit_ids=None, 
                         order_channel_by_depth=False, 
                         time_range=None, 
                         unit_colors=None, 
                         sparsity=None, 
                         mode='auto', 
                         return_scaled=False, 
                         cmap='RdBu', 
                         show_channel_ids=False, 
                         color_groups=False, 
                         color=None, 
                         clim=None, 
                         tile_size=512, 
                         seconds_per_row=0.2, 
                         scale=1, 
                         spike_width_ms=4, 
                         spike_height_um=20, 
                         with_colorbar=True, 
                         backend="ipywidgets")


## Unit quality metrics

In [ ]:
from spikeinterface.postprocessing import compute_principal_components
from spikeinterface.qualitymetrics import compute_quality_metrics, get_quality_metric_list

In [ ]:
get_quality_metric_list()

In [ ]:
metrics = compute_quality_metrics(analyzer, metric_names=["snr", "isi_violation", "nearest_neighbor", "firing_rate", 'presence_ratio', 'amplitude_cutoff'])

In [ ]:
print (metrics)

In [ ]:
keep_mask = (metrics["amplitude_cutoff"] < 1e-6)
print(keep_mask)

In [ ]:
keep_unit_ids = keep_mask[keep_mask].index.values
keep_unit_ids = [unit_id for unit_id in keep_unit_ids]
print(keep_unit_ids)

In [ ]:
from scipy.ndimage.filters import gaussian_filter1d
plt.rcParams.update({'font.size': 14})

def plot_metric(data, bins, x_axis_label, color, max_value=-1):
    
    h, b = np.histogram(data, bins=bins, density=True)

    x = b[:-1]
    y = gaussian_filter1d(h, 1)

    plt.plot(x, y, color=color)
    plt.xlabel(x_axis_label)
    plt.gca().get_yaxis().set_visible(False)
    [plt.gca().spines[loc].set_visible(False) for loc in ['right', 'top', 'left']]
    if max_value < np.max(y) * 1.1:
        max_value = np.max(y) * 1.1
    plt.ylim([0, max_value])
    
    return max_value

In [ ]:
ss.get_default_sorter_params('spykingcircus2')


In [ ]:
sorter_spykingcircus2 = ss.run_sorter(
                sorter_name='spykingcircus2',
                recording = rec_fil,
                verbose=True,
                folder = 'output/spykingcircus2',
                remove_existing_folder=True  ## CUIDADO, SOBREESCRIBE LOS DATOS EN CASO DE HABER UNA CARPETA, PARA DESHABILITAR PONER =FALSE
                )

## Recrear experimento (en Desarrollo)

In [ ]:
log_file = "logs/spike_sorting.log"
experiment_data = recreate_experiment(log_file)

if experiment_data:
    # Cargar grabaciones
    recording = read_rhd(Path(experiment_data["rhd_files"][0]).parent)
    probegroup = pi.read_probeinterface(experiment_data["probegroup_file"])
    
    # Aplicar procesamiento
    recording = prep.bandpass_filter(recording, freq_min=500., freq_max=9000.)
    recording = recording.set_probegroup(probegroup, group_mode='by_probe')
    
    if "triggers" in experiment_data:
        recording = prep.remove_artifacts(
            recording=recording,
            list_triggers=experiment_data["triggers"],
            ms_after=500,
            mode="zeros"
        )
    
    print("Experimento recreado exitosamente.")
else:
    print("No se pudo recrear el experimento.")
